OpenStreetMap is an open project, which means it's free and everyone can use it and edit as they like. OpenStreetMap is direct competitor of Google Maps. How OpenStreetMap can compete with the giant you ask? It's depend completely on crowd sourcing. There's lot of people willingly update the map around the world, most of them fix their map country. 

Openstreetmap is so powerful, and rely heavily on the human input. But its strength also the downfall. Everytime there's human input, there's always be human error.It's very error prone.
<!-- TEASER_END -->

# Problems Encountered in the Map

Take the name of the street for example. People like to abbreviate the type of the street.
Street become St. st. In Indonesia, 'Jalan'(Street-Eng), also abbreviated as Jln, jln, jl, Jln.
It maybe get us less attention. But for someone as Data Scientist/Web Developer, they expect the street to have generic format. 

    'Jalan Sudirman' -> Jalan <name> -> name = Sudirman
    'Jln Sudirman' -> Jalan <name> -> ERROR!
    
This project tends to fix that, it fix abbreviate name, so it can use more generalize type. Not only it's benefit for professional, But we can also can see more structured words. 

In this project, i want to show you to fix one type of error, that is the address of the street.
I choose whole places of Jakarta. Jakarta is the capital of Indonesia.This dataset is huge, over 250,000 examples. It's my hometown, and i somewhat want to help the community.
And not only that, i also will show you how to put the data that has been audited into MongoDB instance. We also use MongoDB's Agregation Framework to get overview and analysis of the data.

In [4]:
OSM_FILE = 'dataset/jakarta.osm'

In [ ]:
%load 02-codes/mapparser.py

To audit the osm file, first we need to know the overview of the data.
To get an overview of the data, we count the tag content of the data.

In [ ]:
%load 02-codes/users.py

In [2]:
import xml.etree.cElementTree as ET

In [ ]:
%load 02-codes/audit.py

This will save the jakarta osm that has been audited into jakarta_audit.osm
Not let's prepare the audited file to be input to the MongoDB instance.

In [ ]:
# %load 02-codes/data.py
#!/usr/bin/env python
import xml.etree.ElementTree as ET
import pprint
import re
import codecs
import json


lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
addresschars = re.compile(r'addr:(\w+)')
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
OSM_FILE = 'jakarta_audit.osm'

def shape_element(element):
    #node = defaultdict(set)
    node = {}
    if element.tag == "node" or element.tag == "way" :
        #create the dictionary based on exaclty the value in element attribute.
        node = {'created':{}, 'type':element.tag}
        for k in element.attrib:
            try:
                v = element.attrib[k]
            except KeyError:
                continue
            if k == 'lat' or k == 'lon':
                continue
            if k in CREATED:
                node['created'][k] = v
            else:
                node[k] = v
        try:
            node['pos']=[float(element.attrib['lat']),float(element.attrib['lon'])]
        except KeyError:
            pass
        
        if 'address' not in node.keys():
            node['address'] = {}
        #Iterate the content of the tag
        for stag in element.iter('tag'):
            #Init the dictionry

            k = stag.attrib['k']
            v = stag.attrib['v']
            #Checking if indeed prefix with 'addr' and no ':' afterwards
            if k.startswith('addr:'):
                if len(k.split(':')) == 2:
                    content = addresschars.search(k)
                    if content:
                        node['address'][content.group(1)] = v
            else:
                node[k]=v
        if not node['address']:
            node.pop('address',None)
        #Special case when the tag == way,  scrap all the nd key
        if element.tag == "way":
            node['node_refs'] = []
            for nd in element.iter('nd'):
                node['node_refs'].append(nd.attrib['ref'])
#         if  'address' in node.keys():
#             pprint.pprint(node['address'])
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    """
    Process the osm file to json file to be prepared for input file to monggo
    """
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

def test():

    data = process_map(OSM_FILE)
    pprint.pprint(data[500])


if __name__ == "__main__":
    test()

The processed map has ben saved to jakarta_audit.osm.json
Now that we have process the audited map file into array of JSON, let's put it into mongodb instance.
this will take the map that we have been audited.
First we load the script to insert the map

In [2]:
from data import *
import pprint

In [4]:
data = process_map('dataset/jakarta_audit.osm')

Okay let's test if the data is something that we expect

In [6]:
pprint.pprint(data[0:2])

[{'created': {'changeset': '20029239',
              'timestamp': '2014-01-16T08:18:23Z',
              'uid': '646006',
              'user': 'Irfan Muhammad',
              'version': '13'},
  'id': '29938967',
  'pos': [-6.1803929, 106.8226699],
  'type': 'node'},
 {'created': {'changeset': '20029239',
              'timestamp': '2014-01-16T08:18:23Z',
              'uid': '646006',
              'user': 'Irfan Muhammad',
              'version': '28'},
  'id': '29938968',
  'pos': [-6.1803972, 106.8231199],
  'type': 'node'}]


The data seems about right.
After we verified the data is ready, let's put it into MongoDB

In [7]:
from pymongo import MongoClient

In [8]:
client  = MongoClient('mongodb://localhost:27017')
db = client.examples

In [9]:
[db.jktosm.insert(e) for e in data]

/Users/Jon/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  if __name__ == '__main__':


ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

Okay, it seems that we have sucessfully insert all of our data into MongoDB instance.
Let's test this

In [ ]:
pipeline = [
    {'$limit' : 3}
]
pprint.pprint(db.jktosm.aggregate(pipeline)['result'])

# Overview of the data

You can see the filesize about the dataset.

In [1]:
!ls -lh dataset/jakarta*

-rwxrwxrwx@ 1 Jon  staff    79M Nov  5  2014 dataset/jakarta.osm
-rwxrwxrwx  1 Jon  staff    80M Nov 20  2014 dataset/jakarta_audit.osm
-rwxrwxrwx  1 Jon  staff    89M Nov 20  2014 dataset/jakarta_audit.osm.json


### Show 5 data that have street 

In [7]:
pipeline = [
            {'$match': {'address.street':{'$exists':1}}},
            {'$limit' : 5}
]
result  = db.jktosm.aggregate(pipeline)['result']
pprint.pprint(result)

[{u'_id': ObjectId('546d9d758cbd2f060eb3916d'),
  u'address': {u'housename': u'Pasar Festival',
               u'street': u'Jalan HR Rasuna Said'},
  u'building': u'yes',
  u'created': {u'changeset': u'16848088',
               u'timestamp': u'2013-07-06T12:21:11Z',
               u'uid': u'76518',
               u'user': u'Firman Hadi',
               u'version': u'2'},
  u'id': u'1394516071',
  u'leisure': u'sports_centre',
  u'name': u'Soemantri Brojonegoro',
  u'pos': [-6.2213611, 106.8329498],
  u'sport': u'basketball',
  u'type': u'node'},
 {u'_id': ObjectId('546d9d768cbd2f060eb39c64'),
  u'address': {u'city': u'Jakarta',
               u'country': u'ID',
               u'housename': u'Meruvian Camp - Cempaka Baru',
               u'housenumber': u'39',
               u'street': u'Jalan Swadaya 2 No. 39'},
  u'created': {u'changeset': u'9758314',
               u'timestamp': u'2011-11-06T18:44:31Z',
               u'uid': u'70696',
               u'user': u'xybot',
              

### Show the top 5 of contributed users

We also can find the top 5 contributed users. These users are count by how they created the point in the map, and sort descent 

In [45]:
pipeline = [
            {'$match': {'created.user':{'$exists':1}}},
            {'$group': {'_id':'$created.user',
                        'count':{'$sum':1}}},
            {'$sort': {'count':-1}},
            {'$limit' : 5}
]
result  = db.jktosm.aggregate(pipeline)['result']
pprint.pprint(result)

[{u'_id': u'Firman Hadi', u'count': 113770},
 {u'_id': u'dimdim02', u'count': 38860},
 {u'_id': u'riangga_miko', u'count': 36695},
 {u'_id': u'raniedwianugrah', u'count': 30388},
 {u'_id': u'Alex Rollin', u'count': 26496}]


### Show the restaurant's name, the food they serve, and contact number

In [9]:
pipeline = [
            {'$match': {'amenity':'restaurant',
                        'name':{'$exists':1}}},
            {'$project':{'_id':'$name',
                         'cuisine':'$cuisine',
                         'contact':'$phone'}}
]
result  = db.jktosm.aggregate(pipeline)['result']
pprint.pprint(result)

[{u'_id': u'Taman Hek'},
 {u'_id': u'3 House'},
 {u'_id': u'Jimbaran'},
 {u'_id': u'Death by Chocolate'},
 {u'_id': u"McDonald's"},
 {u'_id': u"Chef's Kitchen"},
 {u'_id': u'Planet Hollywood Jakarta', u'cuisine': u'american'},
 {u'_id': u'Soto kudus'},
 {u'_id': u'KFC Cikini', u'cuisine': u'chicken'},
 {u'_id': u'Mc Donald Cikini'},
 {u'_id': u'Pempek Cuko'},
 {u'_id': u'Warung Tekko',
  u'contact': u'+62 21 5263137',
  u'cuisine': u'Indonesian'},
 {u'_id': u'Kafe Betawi', u'cuisine': u'asian'},
 {u'_id': u'QBox Cafe', u'cuisine': u'asian'},
 {u'_id': u'Comics Cafe', u'cuisine': u'american'},
 {u'_id': u'Pizza Hut', u'cuisine': u'pizza'},
 {u'_id': u'Otel Lobby', u'cuisine': u'international'},
 {u'_id': u'Loewy', u'cuisine': u'french'},
 {u'_id': u'Food Court Passer Kuningan', u'cuisine': u'asian'},
 {u'_id': u'Pastis', u'cuisine': u'italian'},
 {u'_id': u'Pizza Hut', u'cuisine': u'pizza'},
 {u'_id': u'Dunkin donuts'},
 {u'_id': u'Warung Pasta'},
 {u'_id': u'Ayam Balphuss'},
 {u'_id': 

# Other ideas about the datasets

Earlier, we have found that not only most of the public places can be found, but it also  shared what kind of description about these places. Now that we have those data, we can create mobile apps that serve as an assistant where to point the users to their need. 

Mobile apps can be opened as a simple text box, and ask the users *"What do you want?"*. If for example, the users say *"I want to eat Japanese food"*, the assistant will find Japanese cuisine restaurant within 1 mile radius. Or if the users ask *"I want to eat chicken (Indonesia: Ayam)"*, the assistant will find restaurant's name with 'Ayam' in it. They will provide these restaurants based on the map location, with user location at the exact center of the map. If the user click on one restaurant, it will be given some pop up that give the description of the restaurant. For example, their kind of cuisine, contact number, and ask users whether they want to be given direction to the restaurant. 

Not only restaurant, but other kind of public places. Watch movies, go to gym center, finding a flower, etc. This is a lot of possibilities.

I actually submit the changes that I made back to the OSM. The changeset is here: http://osmhv.openstreetmap.de/changeset.jsp?id=26730562

But the changes get reverted back, because I violated some rules that stated I can't change the map with machine code. If you see the changes, I actually made a lot of changes that the community will benefit.